In [1]:
# pip install pandas pandas_datareader

Note: you may need to restart the kernel to use updated packages.


'c:\program' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
#first importing all the necessary libraries that we need
import math
# import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense,LSTM
import yfinance as yf
plt.style.use('fivethirtyeight')


AttributeError: module 'tensorflow.python.training.experimental.mixed_precision' has no attribute '_register_wrapper_optimizer_cls'

In [ ]:
# Get the stock quote i.e the stock data
# df=web.DataReader('YESBANK ',data_source="yahoo",start='2008-01-01',end='2020-08-31')
# df = yf.download('AAPL', start='2015-01-01',end='2021-08-01')
df = yf.download('AAPL', period='5y')
print(df)

In [ ]:
# Exploring the data
print(df.head())
print(df.tail())

In [ ]:
# The no. of rows and columns in the dataset
df.shape

In [ ]:
# Visualing the data

plt.figure(figsize=(16,8))
plt.title("Closing price history")
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close price USD($)',fontsize=18)
plt.show()

In [ ]:
# creating a new dataframe with only the "Close" column
data=df.filter(['Close'])
dataset=data.values
dataset

In [ ]:
# Selecting the no. of rows to train the model on
training_size=math.ceil(len(dataset)*0.8)
training_size

In [ ]:
#scaling the data 
# It will make our data in the range of [0,1]
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

In [ ]:
#creating the training dataset i.e, scaled training dataset
train_data=scaled_data[:training_size,:]

#rest data will be our testing dataset
test_data=scaled_data[training_size-60:,:]

train_data.shape,test_data.shape

In [ ]:
# Split the data into x_train and y_train datasets
# i.e, independent and dependent training features
x_train=[]
y_train=[]
for i in range(60,training_size):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if(i<=61):
        print(x_train)
        print(y_train)
        print()

In [ ]:
#convert the x_train and y_train to numpy arrays
x_train,y_train=np.array(x_train),np.array(y_train)

In [ ]:
#reshape the data to 3-dimensional since our model requires 3d data values
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
#build the LSTM model
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
#compile the model
model.compile(optimizer='adam',loss='mean_squared_error')
# Train the model
model.fit(x_train,y_train,batch_size=60,epochs=5)

In [ ]:
#create the testing dataset
x_test=[]
# y_test=dataset[training_]
y_test=dataset[training_size:,:]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])


In [ ]:
#convert the data to a numpy array
x_test=np.array(x_test)
#reshaping the data to be 3-d
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
x_test.shape

In [ ]:
#get the models predicted price values
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)

In [ ]:
#get the root mean squared error(RMSE)
rmse=np.sqrt(np.mean(((predictions-y_test)**2)))
rmse
# rmse=np.sqrt(((predictions-y_test)**2),mean())


In [ ]:
#plot the data
train=data[:training_size]
valid=data[training_size:]
valid['Predictions']=predictions

In [ ]:
#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close price USD($)',fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Valid','Predictions'],loc='lower right')
plt.show()

In [ ]:
#the numerical data
valid

In [ ]:
# Now lets see our model is working properly by predicting the closing stock of one day ahead
#get the quote
# apple_quote=web.DataReader('AAPL',data_source='yahoo',start='2021-06-25',end='2021-07-01')
apple_quote=yf.download('AAPL',period='10d')
#create a new dataframe
new_df=apple_quote.filter(['Close'])
print(new_df)

In [ ]:
#now taking the last 60 day closing price values and converting the dataframe
# to an ARRAY
last_60=test_data[-60:]
last_60

In [ ]:
# now scaling the data to be between 0 and 1
# last_60_scaled=scaler.transform(last_60)
last_60_scaled=test_data[-60:]
last_60_scaled

In [ ]:
# Now creating a dataset of this last 60 day
X_test=[last_60_scaled]
#Converting it into numpy array
X_test=np.array(X_test)
#reshaping the array to be 3 dimensional
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
#Now calculating the predicted scale price
pred=model.predict(X_test)
print(pred)
#undoing the scaling
pred=scaler.inverse_transform(pred)
print(pred)

In [ ]:
print(new_df)

In [ ]:
# as we could see above after 25 june theres the 28th june so it is giving 134-799 valid closing price 
# and we got 133.183 which is very close so our model's prediction is very close we can increase it more
# further if we increate the test data or increase the model epoch

In [ ]:
# Now lets check if the predicted values match with the actual value or not
# apple_quote2=web.DataReader('AAPL',data_source='yahoo',start='2021-01-01',end='2021-01-04')
# apple_quote2

In [ ]:
# apple_quote2['Close']
# as we can see the actual price is 129 which is very close to our predicted price 123

## Predicting next 10 days

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[-60:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
temp_input

In [ ]:
# demonstrate prediction for next 10 days
from numpy import array

lst_output=[]
n_steps=60
i=0
while(i<10):
    
    if(len(temp_input)>60):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
lst_output=np.array(lst_output)
print(scaler.inverse_transform(lst_output))

In [ ]:
day_new=np.arange(1,61)
day_pred=np.arange(61,71)

In [ ]:
print(len(scaled_data),scaled_data)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(scaled_data[-60:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df3=scaled_data.tolist()
df3.extend(lst_output)
plt.plot(df3[-100:])

In [ ]:
apple_quote3=web.DataReader('AAPL',data_source='yahoo',start='2021-05-28',end='2021-06-30')
apple_quote3['Close']

In [ ]:
scaler.inverse_transform(lst_output)